<a href="https://colab.research.google.com/github/Hetul803/finance-research-assistant/blob/main/notebooks/phi3_student_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers>=4.40.1 accelerate>=0.27.2


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
from transformers import pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

In [ ]:
messages = [
             {"role":"system","content":"Your are a helpful AI assisant."},
             {"role":"user","content":"Who is salman khan?"},
]
output = generator(messages)
print(output[0]["generated_text"])

In [ ]:
from transformers import pipeline
chatbot = pipeline("text-generation",model=model, tokenizer=tokenizer)


In [ ]:
# Default: deterministic (no sampling, so no temperature)
GEN_ARGS = {
    "max_new_tokens": 200,
    "return_full_text": False,
    "do_sample": False,
    # (no temperature/top_p keys when do_sample=False)
}

def set_sampling(on: bool, temperature: float = 0.7, top_p: float = 0.9):
    """Turn sampling on/off and set temperature/top_p without reloading the model."""
    GEN_ARGS["do_sample"] = bool(on)
    if on:
        GEN_ARGS["temperature"] = float(temperature)
        GEN_ARGS["top_p"] = float(top_p)
    else:
        # Remove keys so HF won't warn about ignored args
        GEN_ARGS.pop("temperature", None)
        GEN_ARGS.pop("top_p", None)

# Example: turn ON temperature now (adjust values anytime)
set_sampling(on=True, temperature=0.7, top_p=0.9)


In [ ]:
HISTORY = []  # list of (user, assistant) tuples
SYSTEM = "You are a helpful assistant."

def chat(message: str):
    """Ask a question, keep conversation context, use current GEN_ARGS."""
    # Build messages from history
    messages = [{"role": "system", "content": SYSTEM}]
    for u, a in HISTORY:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": message})

    # Generate with the already-loaded pipeline + current GEN_ARGS
    out = chatbot(messages, **GEN_ARGS)
    answer = out[0]["generated_text"]

    # Save to history and print
    HISTORY.append((message, answer))
    print(answer)

def clear_history():
    HISTORY.clear()


In [ ]:
# Temperature is ON right now from Step 2 (0.7). Ask anything:
chat("Who is Gandhi?")
chat("Explain attention very simply.")


In [ ]:
chat("Explain algorithms.")



In [ ]:
chat("Explain quantum computing.")